# Data preprocessing

The data exploration step in the previous notebook already showed that there are some missing values in the data set and some which are not in a good format for furhter analysis. Since the attributes of the data set are well known it is possible to filter out semantic inconsistencies. There might for example be records with the same departure and arrival station.

The goal of this step is to handle missing values, transform some values and export a clean data set which can be used for further analysis.

## Renaming columns

Firstly the columns will be renamed to have english labels as defined in the attribute description.

In [ ]:
import pandas as pd

# The first three lines are invalid -> skip
journeys = pd.read_csv('../data/train-drives.csv', skiprows=3, encoding='utf-8')

# Rename columns, optional
journeys.rename(columns={
    'Lfd. Nummer': 'ID',
    'Linien Nummer': 'train_line',
    'Start Haltestelle ': 'departure_station',
    'Ziel Haltestelle': 'arrival_station',
    'Planmäßige Abfahrtszeit': 'planned_departure',
    'Datum': 'date',
    'Verspätung in min': 'delay',
    'Kontrolliert': 'ticket_checked',
    'Gleis verlegt': 'platform_changed',
    'Fülle des Zuges': 'crowdedness',
    'Zugmodel': 'train_model',
    'Sauberkeit': 'cleanliness',
    'rel. planmäßige Abfahrtszeit': 'relative_planned_departure',
    'rel. Verspätung': 'relative_delay',
    'Alternativer Anschluss': 'alternative_connection',
}, inplace=True)

# Drop unamed columns
journeys = journeys.loc[:, ~journeys.columns.str.contains('^Unnamed')]
# Drop unnecessary columns
journeys.drop(columns=['ID', 'Bemerkung', 'Definition Verspätung: '], inplace=True)

# Calculate actual length for later evaluation
org_len = len(journeys)
non_boolean_cols = ['train_line', 'departure_station', 'arrival_station', 'planned_departure',
                    'date', 'delay', 'crowdedness', 'train_model', 'cleanliness',
                    'relative_planned_departure', 'relative_delay']
journeys.dropna(how='all', subset=non_boolean_cols, inplace=True)
non_empty_len = len(journeys)

journeys.head(5)


In [ ]:
# Remove rows with uninteresting values

# Drop all records with rare departure or arrival stations
rastede_bremen_stations = ['Rastede', 'Oldenburg', 'Hude', 'Delmenhorst', 'Bremen', 'Bremen-Neustadt']
journeys = journeys[
    journeys['departure_station'].isin(rastede_bremen_stations)
    & journeys['arrival_station'].isin(rastede_bremen_stations)
]

# Remove all records where delay is 'X' or 'N', case-insensitive
journeys = journeys[
    ~journeys['delay'].astype(str).str.upper().isin(['X', 'N'])
]


In [ ]:
# Data conversion and encoding

# One-Hot encoding for train line
journeys = pd.get_dummies(journeys, columns=['train_line'], prefix='train_line')

# Remove invalid times for planned departure
journeys['planned_departure'] = pd.to_datetime(journeys['planned_departure'], format='%H:%M', errors='coerce')
journeys = journeys.dropna(subset=['planned_departure'])
# Convert time into an hour column and group by 2 hour blocks
journeys['planned_departure_hour'] = journeys['planned_departure'].dt.hour
journeys['planned_departure_hour'] = (journeys['planned_departure_hour'] // 2) * 2

# Convert date to datetime and remove invalid dates
journeys['date'] = pd.to_datetime(journeys['date'], format='%d.%m.%Y', errors='coerce')
journeys.dropna(subset=['date'], inplace=True)

# Extract day of the week (Monday=1, Sunday=7) and month (1-12)
journeys['day_of_week'] = journeys['date'].dt.isocalendar().day
journeys['month'] = journeys['date'].dt.month

# Convert delay to integer
delay_numeric = pd.to_numeric(journeys['delay'], errors='coerce')
journeys['delay'] = delay_numeric.fillna(0).astype(int)

# Apply binary encoding for ticket_checked
journeys['ticket_checked'] = journeys['ticket_checked'].astype(bool).astype(int)

# Encode crowdedness with ordinal encoding
journeys = journeys[
    # Remove empty values
    journeys['crowdedness'].astype('str').str.strip() != ''
]
crowdedness_mapping = {
    '(Fast) Leer': 0,
    'Wenig voll': 1,
    'Normal voll': 2,
    'Sehr voll': 3,
    'Zu voll': 4
}
journeys['crowdedness'] = pd.to_numeric(journeys['crowdedness'].map(crowdedness_mapping), errors='raise', downcast='integer')


In [ ]:
# Compare org_len to current length
new_len = len(journeys)
print(f'Original length (with empty records): {org_len}')
print(f'Original length (without empty records): {non_empty_len}, New length: {new_len}, Removed: {non_empty_len - new_len}')

# Save the cleaned data to a new CSV file
journeys.to_csv('../data/train-drives-cleaned.csv', index=False, encoding='utf-8')
journeys.head(5)
